# Quantum Phase Estimation: Experiments and Analysis

This notebook demonstrates both textbook and iterative QPE on various unitary operators.

**What we'll cover:**
1. Estimating the T gate phase ($\phi = 1/8$) with textbook QPE
2. How precision scales with the number of ancilla qubits
3. Iterative QPE: single-ancilla approach with classical feedback
4. Comparing textbook vs iterative QPE on custom phases

The T gate is the standard benchmark because its phase (1/8) is exactly representable in 3 binary digits, so we can verify correctness exactly.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.textbook_qpe import run_qpe, precision_sweep, build_qpe_circuit
from src.iterative_qpe import iterative_phase_estimation, iterative_precision_sweep
from src.unitaries import (
    t_gate_unitary, phase_gate, custom_unitary,
    eigenstate_circuit_one, get_true_phase
)
from src.plotting import (
    plot_phase_histogram, plot_precision_vs_ancilla,
    plot_iterative_convergence, plot_method_comparison
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

print('Setup complete!')

## 1. T Gate Phase Estimation

The T gate has eigenvalues $\{1, e^{i\pi/4}\}$. For the $|1\rangle$ eigenstate, the eigenphase is $\phi = 1/8 = 0.125$.

In binary, $0.125 = 0.001_2$, so we need exactly 3 ancilla qubits to represent this phase perfectly. Let's verify this and look at the measurement distribution.

In [ ]:
# T gate QPE
U_t = t_gate_unitary()
true_phase = get_true_phase(U_t, eigenstate='|1>')
print(f"T gate unitary:\n{np.round(U_t, 4)}")
print(f"True eigenphase: {true_phase} = {true_phase} (should be 0.125)\n")

# run QPE with different ancilla counts
print("=== Textbook QPE: T gate ===")
for n_anc in [2, 3, 4, 5]:
    result = run_qpe(U_t, n_ancilla=n_anc, shots=4096, seed=42)
    print(f"n_ancilla={n_anc}: estimated={result['estimated_phase']:.6f}, "
          f"error={result['phase_error']:.6f}")

# detailed look at n=4
result_4 = run_qpe(U_t, n_ancilla=4, shots=4096, seed=42)
plot_phase_histogram(result_4['counts'], true_phase, n_ancilla=4,
                      save_dir=str(RESULTS_DIR))

# show the measurement distribution
print(f"\nMeasurement counts (n_ancilla=4):")
for bitstring, count in sorted(result_4['counts'].items(),
                                 key=lambda x: -x[1])[:5]:
    print(f"  {bitstring}: {count} ({count/4096:.3f})")

## 2. Precision Scaling

How does the estimation error decrease as we add more ancilla qubits? For a phase that's exactly representable in $n$ bits, the error should drop to zero at $n$. For non-dyadic phases, the error should decrease as $O(1/2^n)$.

Let's test both cases:
- T gate phase = 1/8 (exactly 3 bits)
- Custom phase = 1/3 (not exactly representable in any finite number of bits)

In [ ]:
ancilla_range = range(2, 9)

# T gate (exact phase)
print("=== Precision sweep: T gate (phase = 1/8) ===")
t_results = precision_sweep(U_t, ancilla_range, shots=4096, seed=42)
plot_precision_vs_ancilla(t_results, save_dir=str(RESULTS_DIR))

# custom phase 1/3
print("\n=== Precision sweep: phase = 1/3 ===")
U_third = custom_unitary(1/3)
third_results = precision_sweep(U_third, ancilla_range, shots=4096, seed=42)

# plot both
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, results, title in [(axes[0], t_results, 'T gate ($\\phi = 1/8$)'),
                            (axes[1], third_results, 'Custom ($\\phi = 1/3$)')]:
    n_vals = results['n_ancilla']
    errors = [e + 1e-15 for e in results['errors']]
    theoretical = [1.0 / (2 ** n) for n in n_vals]

    ax.semilogy(n_vals, errors, 'o-', color='#FF6B6B',
                linewidth=2, markersize=8, label='Actual Error')
    ax.semilogy(n_vals, theoretical, 's--', color='#4ECDC4',
                linewidth=1.5, markersize=6, alpha=0.7,
                label='$1/2^n$ bound')
    ax.set_xlabel('Number of Ancilla Qubits')
    ax.set_ylabel('Error (log scale)')
    ax.set_title(title)
    ax.legend()
    ax.set_xticks(n_vals)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'precision_comparison.png', dpi=150)
plt.show()

print("\nNotice: T gate error drops to machine precision at n=3,")
print("while 1/3 phase error decreases gradually.")

## 3. Iterative QPE

The iterative approach uses only 1 ancilla qubit. It estimates the phase one bit at a time, using classical feedback to correct the phase at each step.

Let's run iterative QPE on the T gate and watch the phase estimate converge.

In [ ]:
print("=== Iterative QPE: T gate ===")
iter_result = iterative_phase_estimation(U_t, n_iterations=6,
                                          shots=4096, seed=42)

print(f"Estimated phase: {iter_result['estimated_phase']:.6f}")
print(f"True phase: {iter_result['true_phase']:.6f}")
print(f"Error: {iter_result['phase_error']:.6f}")
print(f"Confidence: {iter_result['confidence']:.3f}")
print(f"Bit estimates: {iter_result['bit_estimates']}")

# plot convergence
plot_iterative_convergence(iter_result['phase_estimates'],
                           iter_result['true_phase'],
                           save_dir=str(RESULTS_DIR))

print("\nPhase estimate after each iteration:")
for i, est in enumerate(iter_result['phase_estimates']):
    error = min(abs(est - true_phase), abs(est - true_phase + 1))
    print(f"  Iteration {i + 1}: phase = {est:.6f}, error = {error:.6f}")

## 4. Textbook vs Iterative Comparison

Both methods should achieve similar precision for the same number of bits. The tradeoff is:
- **Textbook**: needs $n$ ancilla qubits but is more robust to shot noise
- **Iterative**: needs only 1 ancilla qubit but is more sensitive to measurement errors

Let's compare them head-to-head on a non-trivial phase.

In [ ]:
# compare on custom phase = 0.3
U_custom = custom_unitary(0.3)
bit_range = range(2, 9)

print("=== Textbook QPE: phase = 0.3 ===")
text_results = precision_sweep(U_custom, bit_range, shots=4096, seed=42)

print("\n=== Iterative QPE: phase = 0.3 ===")
iter_results = iterative_precision_sweep(U_custom, bit_range, shots=4096, seed=42)

# comparison plot
plot_method_comparison(text_results, iter_results, save_dir=str(RESULTS_DIR))

# summary table
print(f"\n{'Bits':>5}  {'Textbook Error':>14}  {'Iterative Error':>15}")
print("-" * 40)
for i, n in enumerate(bit_range):
    te = text_results['errors'][i]
    ie = iter_results['errors'][i]
    print(f"{n:>5}  {te:>14.6f}  {ie:>15.6f}")

print(f"\nDone! All plots saved to results/")

## Summary

**Key takeaways:**

1. **QPE works** — for phases that are exactly representable in $n$ bits (like T gate with $n=3$), QPE finds the exact phase with very high probability.

2. **Precision scales exponentially** — each additional ancilla qubit doubles the phase resolution. The error decreases as $O(1/2^n)$ for general phases.

3. **Iterative QPE is practical** — using only 1 ancilla qubit, the iterative approach achieves comparable precision to the textbook version. This makes it much more suitable for near-term quantum hardware.

4. **Shot noise matters** — for non-dyadic phases, the QPE output distribution spreads across multiple bitstrings. More shots help, but eventually the precision is limited by the number of ancilla qubits (or iterations).

5. **The tradeoff is clear** — textbook QPE uses $n$ ancilla qubits but gets all bits simultaneously. Iterative QPE uses 1 ancilla qubit but needs $n$ sequential rounds. On real hardware, the choice depends on available qubits vs coherence time.